# Лабораторная работа №4

## Подготовка входных данных

Импортируем необходимые библиотеки.

In [52]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [53]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [54]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [55]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [56]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [57]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методами наивного Байеса. В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем библиотеку sklearn.naive_bayes для классификации датасета методами наивного Байеса.

In [59]:
import sklearn.naive_bayes

### Гауссовский наивный Байес

Создаём объект класса GaussianNB, реализующий метод гауссовского наивного Байеса.

In [60]:
gaussian_classifier = sklearn.naive_bayes.GaussianNB()

Определяем параметры, которые мы хотим оптимизировать для GaussianNB.
'var_smoothing': Это параметр, который контролирует сглаживание дисперсии признаков в методе Гауссовского наивного Байеса. Помогает избежать переобучения или недообучения модели.

In [61]:
gaussian_params ={
    'var_smoothing': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [62]:
gaussian_grid = GridSearchCV(gaussian_classifier, gaussian_params, cv=5, n_jobs=-1)
gaussian_grid.fit(X_train, y_train)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

Обучаем модель на тренировочных данных.

In [63]:
best_gaussian_model = sklearn.naive_bayes.GaussianNB(**gaussian_grid.best_params_)
best_gaussian_model.fit(X_train, y_train)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [13]:
gaussian_predicted = best_gaussian_model.predict(X_test)

Вывод отчёта.

In [14]:
print('Параметры:', gaussian_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, gaussian_predicted, digits=5))

Параметры: {'var_smoothing': np.float64(0.7924828983539186)}
Оценка:
               precision    recall  f1-score   support

       False    0.82002   0.97561   0.89107      9060
        True    0.73969   0.24455   0.36757      2568

    accuracy                        0.81416     11628
   macro avg    0.77986   0.61008   0.62932     11628
weighted avg    0.80228   0.81416   0.77546     11628



### Мультиномиальный наивный Байес

Создаём объект класса MultinomialNB, реализующий метод мультиноманального наивного Байеса.

In [15]:
classifier = sklearn.naive_bayes.MultinomialNB()

Определяем параметры, которые мы хотим оптимизировать для MultinomialNB.
'alpha': Это параметр, который контролирует сглаживание в модели мультиномиального наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [16]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [17]:
multinomial_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
multinomial_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [18]:
best_model = sklearn.naive_bayes.MultinomialNB(**multinomial_grid.best_params_)
best_model.fit(X_train, y_train)

MultinomialNB(alpha=np.float64(1e-10))

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [19]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [20]:
print('Параметры:', multinomial_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': np.float64(1e-10)}
Оценка:
               precision    recall  f1-score   support

       False    0.78038   0.99934   0.87639      9060
        True    0.76923   0.00779   0.01542      2568

    accuracy                        0.78036     11628
   macro avg    0.77481   0.50356   0.44591     11628
weighted avg    0.77792   0.78036   0.68625     11628



### Комплементарный наивный Байес

Создаём объект класса ComplementNB, реализующий метод комплементарного наивного Байеса.

In [21]:
classifier = sklearn.naive_bayes.ComplementNB()

Определяем параметры, которые мы хотим оптимизировать для ComplementNB.
'alpha': Это параметр, который контролирует сглаживание в модели комплементарного наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [22]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [23]:
complement_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
complement_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07,...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [24]:
best_model = sklearn.naive_bayes.ComplementNB(**complement_grid.best_params_)
best_model.fit(X_train, y_train)

ComplementNB(alpha=np.float64(13848.863713938747))

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [25]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [26]:
print('Параметры:', complement_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': np.float64(13848.863713938747)}
Оценка:
               precision    recall  f1-score   support

       False    0.81238   0.94581   0.87403      9060
        True    0.54537   0.22936   0.32292      2568

    accuracy                        0.78758     11628
   macro avg    0.67888   0.58758   0.59847     11628
weighted avg    0.75341   0.78758   0.75232     11628



### Бернуллиевский наивный Байес

Создаём объект класса BernoulliNB, реализующий метод бернулливского наивного Байеса.

In [27]:
classifier = sklearn.naive_bayes.BernoulliNB()

Определяем параметры, которые мы хотим оптимизировать для BernoulliNB.
'alpha': Это параметр, который контролирует сглаживание в модели бернуллиевского наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.

In [28]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [29]:
bernoulli_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
bernoulli_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07, 4...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10])})

Обучаем модель на тренировочных данных.

In [30]:
best_model = sklearn.naive_bayes.BernoulliNB(**bernoulli_grid.best_params_)
best_model.fit(X_train, y_train)

BernoulliNB(alpha=np.float64(3430.469286314912))

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [31]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [32]:
print('Параметры:', bernoulli_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': np.float64(3430.469286314912)}
Оценка:
               precision    recall  f1-score   support

       False    0.77915   1.00000   0.87587      9060
        True    0.00000   0.00000   0.00000      2568

    accuracy                        0.77915     11628
   macro avg    0.38958   0.50000   0.43794     11628
weighted avg    0.60708   0.77915   0.68244     11628



/home/alexandr/STUDY/DPL/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexandr/STUDY/DPL/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexandr/STUDY/DPL/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

### Категориальный наивный Байес

Создаём объект класса CategoricalNB, реализующий метод категориального наивного Байеса.

In [33]:
classifier = sklearn.naive_bayes.CategoricalNB()

Определяем параметры, которые мы хотим оптимизировать для CategoricalNB.
* 'alpha': Это параметр, который контролирует сглаживание в модели бернуллиевского наивного Байеса. Это количество наблюдений, добавляемых к каждому признаку.
* 'min_categories': Этот параметр устанавливает минимальное количество категорий, которое должно быть в каждом признаке, чтобы этот признак использовался в модели.

In [34]:
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100),
    'min_categories':[2]
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [35]:
categorical_grid = GridSearchCV(classifier, params, cv=5, n_jobs=-1)
categorical_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-10, 1.59228279e-10, 2.53536449e-10, 4.03701726e-10,
       6.42807312e-10, 1.02353102e-09, 1.62975083e-09, 2.59502421e-09,
       4.13201240e-09, 6.57933225e-09, 1.04761575e-08, 1.66810054e-08,
       2.65608778e-08, 4.22924287e-08, 6.73415066e-08, 1.07226722e-07,
       1.70735265e-07, 2.71858824e-07...
       2.25701972e+05, 3.59381366e+05, 5.72236766e+05, 9.11162756e+05,
       1.45082878e+06, 2.31012970e+06, 3.67837977e+06, 5.85702082e+06,
       9.32603347e+06, 1.48496826e+07, 2.36448941e+07, 3.76493581e+07,
       5.99484250e+07, 9.54548457e+07, 1.51991108e+08, 2.42012826e+08,
       3.85352859e+08, 6.13590727e+08, 9.77009957e+08, 1.55567614e+09,
       2.47707636e+09, 3.94420606e+09, 6.28029144e+09, 1.00000000e+10]),
                         'min_categories': [2]})

Обучаем модель на тренировочных данных.

In [36]:
best_model = sklearn.naive_bayes.CategoricalNB(**categorical_grid.best_params_)
best_model.fit(X_train, y_train)

CategoricalNB(alpha=np.float64(1e-10), min_categories=2)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [37]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [38]:
print('Параметры:', categorical_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted, digits=5))

Параметры: {'alpha': np.float64(1e-10), 'min_categories': 2}
Оценка:
               precision    recall  f1-score   support

       False    0.78764   0.98046   0.87354      9060
        True    0.49429   0.06737   0.11857      2568

    accuracy                        0.77881     11628
   macro avg    0.64096   0.52392   0.49606     11628
weighted avg    0.72285   0.77881   0.70681     11628

